In [14]:
import numpy as np
import pandas as pd

In [15]:
data = pd.read_csv('Sentiment.csv')
data.head(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [16]:
data = data[['text','sentiment']]
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [17]:
data = data[data.sentiment != "Neutral"]
data['sentiment']= pd.get_dummies(data['sentiment'], drop_first = True)
data = data.reset_index(drop=True)
data.head()

,text,sentiment
0,RT @ScottWalker: Didn't catch the full #GOPdeb...,True
1,RT @RobGeorge: That Carly Fiorina is trending ...,True
2,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,True
3,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",True
4,RT @warriorwoman91: I liked her and was happy ...,False


In [37]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [22]:
corpus = []

for i in range(0, data.shape[0]):
    tweet = re.sub('[^a-zA-Z]', ' ', data['text'][i])
    tweet = tweet.lower()
    tweet = word_tokenize(tweet)
    
    # Reduce words to their root form
    tweet = [WordNetLemmatizer().lemmatize(w) for w in tweet if not w in set(stopwords.words('english'))]
    
    # Lemmatize verbs by specifying pos
    tweet = [WordNetLemmatizer().lemmatize(w, pos='v') for w in tweet if not w in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

print(corpus[0:3])

['rt scottwalker catch full gopdebate last night scott best line second walker http co zsff', 'rt robgeorge carly fiorina trend hour debate men complete gopdebate say', 'rt danscavino gopdebate w realdonaldtrump deliver highest rat history presidential debate trump http co']


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(corpus)
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
pd.set_option('display.max_rows', None)
df.head(20)

,TF-IDF
zsff,0.453518
catch,0.344600
full,0.330405
second,0.313011
line,0.299403
scottwalker,0.282078
scott,0.274517
best,0.249607
walker,0.235920
last,0.182394


In [25]:
for i in range(0, len(corpus)):
    corpus[i] = re.sub('co','', corpus[i])
    corpus[i] = re.sub('rt','', corpus[i])
    corpus[i] = re.sub('http','', corpus[i])
    

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(corpus)
encoded_docs = tokenizer.texts_to_sequences(corpus)
padded_sequence = pad_sequences(encoded_docs,maxlen=25)

In [29]:
padded_sequence.shape

(10729, 25)

In [30]:
print(tokenizer.word_index['trump'])

4


In [31]:
print(corpus[0])
print(encoded_docs[0])

 sttwalker catch full gopdebate last night stt best line send walker   zsff
[252, 779, 564, 1, 15, 13, 204, 114, 354, 224, 72, 5278]


In [32]:
print(padded_sequence[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0  252
  779  564    1   15   13  204  114  354  224   72 5278]


In [33]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
from keras.initializers import Constant

vocab_size = len(tokenizer.word_index) + 1
embedding_vector_length = 200


model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,
                    input_length=25) )
model.add(SpatialDropout1D(0.2))
model.add(LSTM(3, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 200)           2598000   
                                                                 
 spatial_dropout1d (Spatial  (None, 25, 200)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 3)                 2448      
                                                                 
 dropout (Dropout)           (None, 3)                 0         
                                                                 
 dense (Dense)               (None, 1)                 4         
                                                                 
Total params: 2600452 (9.92 MB)
Trainable params: 2600452 (9.92 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [34]:
# converting the targets to numpy array to feed it into the model
target = np.asarray(data['sentiment'])
print(target)

[ True  True  True ...  True False  True]


In [39]:
MODEL = model.fit(padded_sequence,target,validation_split=0.2, epochs=15, batch_size=256)

Epoch 1/15
34/34 [==============================] - 5s 135ms/step - loss: 0.0618 - accuracy: 0.9857 - val_loss: 0.4531 - val_accuracy: 0.8509
Epoch 2/15
34/34 [==============================] - 5s 138ms/step - loss: 0.0600 - accuracy: 0.9847 - val_loss: 0.4566 - val_accuracy: 0.8439
Epoch 3/15
34/34 [==============================] - 5s 152ms/step - loss: 0.0596 - accuracy: 0.9847 - val_loss: 0.4456 - val_accuracy: 0.8500
Epoch 4/15
34/34 [==============================] - 5s 133ms/step - loss: 0.0586 - accuracy: 0.9851 - val_loss: 0.4576 - val_accuracy: 0.8467
Epoch 5/15
34/34 [==============================] - 5s 146ms/step - loss: 0.0572 - accuracy: 0.9861 - val_loss: 0.4484 - val_accuracy: 0.8472
Epoch 6/15
34/34 [==============================] - 5s 148ms/step - loss: 0.0572 - accuracy: 0.9863 - val_loss: 0.4540 - val_accuracy: 0.8514
Epoch 7/15
34/34 [==============================] - 5s 153ms/step - loss: 0.0577 - accuracy: 0.9850 - val_loss: 0.4473 - val_accuracy: 0.8527
Epoch 

In [36]:
test_word ='Pune is a not a good city'
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=25)
sentiment = int(model.predict(tw).round().item())
#print(sentiment)
if sentiment==0:
    print("Negative")
else:
    print("Positive")


1/1 [==============================] - 0s 406ms/step
Negative
